In [1]:
MODE = "search"
QUERY = ["gg", "taiwan", "btc"]


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.insert(0, "../")

import os
import ray
from attrs import asdict
from ray.util import inspect_serializability
from ray.runtime_env import RuntimeEnv

#
from core.source.crawler import Crawler
from model.source.crawler import CrawlerModel
from core.logger import logger

inspect_serializability(Crawler)

Checking Serializability of <class 'core.source.crawler.Crawler'>


(True, set())

In [4]:
# RUNTIME_ENV = RuntimeEnv(
#     working_dir="/Users/klim/Projects/twitter-trends-crawling/src/"
# )

RUNTIME_ENV = dict(
    working_dir="/Users/klim/Projects/twitter-trends-crawling/src"
)

# ray.init(
#     ignore_reinit_error=True, 
#     runtime_env=RUNTIME_ENV
# )
ray.init()

2022-07-12 01:14:51,337	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265
2022-07-12 01:14:52,834	INFO packaging.py:414 -- Creating a file package for local directory '/Users/klim/Projects/twitter-trends-crawling/src'.
2022-07-12 01:14:52,851	INFO packaging.py:258 -- Pushing file package 'gcs://_ray_pkg_2acc99b08aff1bea.zip' (0.12MiB) to Ray cluster...
2022-07-12 01:14:52,854	INFO packaging.py:267 -- Successfully pushed file package 'gcs://_ray_pkg_2acc99b08aff1bea.zip'.


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.9.6', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-12_01-14-48_376745_55029/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-12_01-14-48_376745_55029/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-07-12_01-14-48_376745_55029', 'metrics_export_port': 65527, 'gcs_address': '127.0.0.1:65476', 'address': '127.0.0.1:65476', 'node_id': '7f8c989fb76ffd59a16a5eacd685f8e831c59474df226d0008d63967'})

In [5]:
from model.source.crawler import CrawlerModel
from core.logger import logger

@ray.remote
def test():    
    model = CrawlerModel(mode="search", query=["gg", "lala"])
    
    # logger.info(sys.path)
    # logger.info(os.getcwd())


    return model.max_limits

tasks = [test.remote() for _ in range(5)]
ray.get(tasks)

RayTaskError(RuntimeError): [36mray::test()[39m (pid=55228, ip=127.0.0.1)
RuntimeError: The remote function failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

[36mray::test()[39m (pid=55228, ip=127.0.0.1)
ModuleNotFoundError: No module named 'model'

2022-07-12 01:19:29,404	ERROR worker.py:94 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::test() (pid=55218, ip=127.0.0.1)
RuntimeError: The remote function failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

ray::test() (pid=55218, ip=127.0.0.1)
ModuleNotFoundError: No module named 'model'
2022-07-12 01:19:29,408	ERROR worker.py:94 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::test() (pid=55221, ip=127.0.0.1)
RuntimeError: The remote function failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

ray::test() (pid=55221, ip=127.0.0.1)
ModuleNotFoundError: No module named 'model'
2022-07-12 01:19:29,412	ERROR worker.py:94 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::test() (pid=55225, ip=127.0.0.1)
RuntimeError: The remote function failed to import on the worker. T

In [4]:
# model=CM(mode=MODE, query=QUERY)
crawler = ray.remote(Crawler)
test = crawler.remote(mode=MODE, query=QUERY)

2022-07-11 22:11:35,662	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265
(TemporaryActor pid=49649) 2022-07-11 22:11:38,928	ERROR worker.py:451 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::Crawler.__init__() (pid=49649, ip=127.0.0.1, repr=<core.source.crawler.FunctionActorManager._create_fake_actor_class.<locals>.TemporaryActor object at 0x1160380a0>)
(TemporaryActor pid=49649) RuntimeError: The actor with name Crawler failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:
(TemporaryActor pid=49649) 
(TemporaryActor pid=49649) ray::Crawler.__init__() (pid=49649, ip=127.0.0.1, repr=<core.source.crawler.FunctionActorManager._create_fake_actor_class.<locals>.TemporaryActor object at 0x1160380a0>)
(TemporaryActor pid=49649) ModuleNotFoundError: No module named 'core'


In [5]:
RUNTIME_ENV = {"py_modules": [CM], "working_dir": "../"}

ray.init(num_cpus=os.cpu_count(), ignore_reinit_error=True, runtime_env=RUNTIME_ENV)

tasks = [test.run.remote()]

data = ray.get(tasks)

2022-07-11 22:11:41,219	INFO worker.py:973 -- Calling ray.init() again after it has already been called.


RayActorError: The actor died because of an error raised in its creation task, [36mray::Crawler.__init__()[39m (pid=49649, ip=127.0.0.1, repr=<core.source.crawler.FunctionActorManager._create_fake_actor_class.<locals>.TemporaryActor object at 0x1160380a0>)
RuntimeError: The actor with name Crawler failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

[36mray::Crawler.__init__()[39m (pid=49649, ip=127.0.0.1, repr=<core.source.crawler.FunctionActorManager._create_fake_actor_class.<locals>.TemporaryActor object at 0x1160380a0>)
ModuleNotFoundError: No module named 'core'

In [10]:
# c = ray.remote(CrawlerModel)

# test = c.remote(mode=CrawlerModel.Mode.SEARCH, query=QUERY)

model = CrawlerModel(mode=MODE, query=QUERY)
test = ray.remote(Crawler)
test = test.remote(mode=CrawlerModel.Mode.SEARCH, query=QUERY)

test

NameError: name 'CrawlerModel' is not defined

In [67]:
from ray.util.iter import from_iterators, from_actors, _ActorSet

a = _ActorSet([test, test], [])

In [68]:
a.actors

[Actor(CrawlerModel, ea51bd9696d8b98733b0a69701000000),
 Actor(CrawlerModel, ea51bd9696d8b98733b0a69701000000)]

In [58]:
it = from_actors([test, test])
it = it.batch_across_shards()

it

LocalIterator[ParallelIterator[from_actors[shards=2]].batch_across_shards()]

LocalIterator[ParallelIterator[from_actors[shards=2]].batch_across_shards()]

In [54]:
next(t)

AttributeError: 'ActorHandle' object has no attribute 'par_iter_init'

In [60]:
type(test)

ray.actor.ActorHandle

In [ ]:
fro